In [ ]:
from datasets import Dataset, DatasetDict
from tqdm import tqdm
import pandas as pd
import json
import wandb
import os

In [ ]:
project_name = "healthyml/In-Context Domain Transfer Improves Out-of-Domain Robustness"
table_paths = {
    # BOSS Sentiment
    # ID
    # BERT
    "BOSS_Sentiment_ID_BERT_Insert": f"",
    "BOSS_Sentiment_ID_BERT_Substitute": f"",
    "BOSS_Sentiment_ID_BERT_Translate": f"",
    "BOSS_Sentiment_ID_BERT_Paraphrase": f"",
    "BOSS_Sentiment_ID_BERT_ICR": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Sentiment_ID_T5_Insert": f"",
    "BOSS_Sentiment_ID_T5_Substitute": f"",
    "BOSS_Sentiment_ID_T5_Translate": f"",
    "BOSS_Sentiment_ID_T5_Paraphrase": f"",
    "BOSS_Sentiment_ID_T5_ICR": f"",

    # Falcon
    "BOSS_Sentiment_ID_Falcon_Insert": f"",
    "BOSS_Sentiment_ID_Falcon_Substitute": f"",
    "BOSS_Sentiment_ID_Falcon_Translate": f"",
    "BOSS_Sentiment_ID_Falcon_Paraphrase": f"",
    "BOSS_Sentiment_ID_Falcon_ICR": f"",

    # SST5
    # BERT
    "BOSS_Sentiment_SST5_BERT_Insert": f"run-hhtbcw8x-sst5_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SST5_BERT_Substitute": f"run-hhtbcw8x-sst5_aug_substitute_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SST5_BERT_Translate": f"run-hhtbcw8x-sst5_aug_backtranslate_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SST5_BERT_Paraphrase": f"run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SST5_BERT_ICR": f"run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Sentiment_SST5_T5_Insert": f"run-hhtbcw8x-sst5_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SST5_T5_Substitute": f"run-hhtbcw8x-sst5_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SST5_T5_Translate": f"run-hhtbcw8x-sst5_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SST5_T5_Paraphrase": f"run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SST5_T5_ICR": f"run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentt5large_style_logs:v0",

    # Falcon
    "BOSS_Sentiment_SST5_Falcon_Insert": f"",
    "BOSS_Sentiment_SST5_Falcon_Substitute": f"",
    "BOSS_Sentiment_SST5_Falcon_Translate": f"",
    "BOSS_Sentiment_SST5_Falcon_Paraphrase": f"",
    "BOSS_Sentiment_SST5_Falcon_ICR": f"",

    # SemEval
    # BERT
    "BOSS_Sentiment_SemEval_BERT_Insert": f"run-hhtbcw8x-semval_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SemEval_BERT_Substitute": f"run-hhtbcw8x-semval_aug_substitute_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SemEval_BERT_Translate": f"run-hhtbcw8x-semval_aug_backtranslate_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SemEval_BERT_Paraphrase": f"run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SemEval_BERT_ICR": f"run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Sentiment_SemEval_T5_Insert": f"run-hhtbcw8x-semval_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SemEval_T5_Substitute": f"run-hhtbcw8x-semval_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SemEval_T5_Translate": f"run-hhtbcw8x-semval_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SemEval_T5_Paraphrase": f"run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SemEval_T5_ICR": f"run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentt5large_style_logs:v0",

    # Falcon
    "BOSS_Sentiment_SemEval_Falcon_Insert": f"",
    "BOSS_Sentiment_SemEval_Falcon_Substitute": f"",
    "BOSS_Sentiment_SemEval_Falcon_Translate": f"",
    "BOSS_Sentiment_SemEval_Falcon_Paraphrase": f"",
    "BOSS_Sentiment_SemEval_Falcon_ICR": f"",

    # Dynasent
    # BERT
    "BOSS_Sentiment_Dynasent_BERT_Insert": f"run-hhtbcw8x-dynasent_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Dynasent_BERT_Substitute": f"run-hhtbcw8x-dynasent_aug_substitute_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Dynasent_BERT_Translate": f"run-hhtbcw8x-dynasent_aug_backtranslate_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Dynasent_BERT_Paraphrase": f"run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Dynasent_BERT_ICR": f"run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Sentiment_Dynasent_T5_Insert": f"run-hhtbcw8x-dynasent_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_Dynasent_T5_Substitute": f"run-hhtbcw8x-dynasent_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_Dynasent_T5_Translate": f"run-hhtbcw8x-dynasent_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_Dynasent_T5_Paraphrase": f"run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_Dynasent_T5_ICR": f"run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentt5large_style_logs:v0",

    # Falcon
    "BOSS_Sentiment_Dynasent_Falcon_Insert": f"",
    "BOSS_Sentiment_Dynasent_Falcon_Substitute": f"",
    "BOSS_Sentiment_Dynasent_Falcon_Translate": f"",
    "BOSS_Sentiment_Dynasent_Falcon_Paraphrase": f"",
    "BOSS_Sentiment_Dynasent_Falcon_ICR": f"",

    # BOSS Toxicity
    # ID
    # BERT
    "BOSS_Toxicity_ID_BERT_Insert": f"",
    "BOSS_Toxicity_ID_BERT_Substitute": f"",
    "BOSS_Toxicity_ID_BERT_Translate": f"",
    "BOSS_Toxicity_ID_BERT_Paraphrase": f"",
    "BOSS_Toxicity_ID_BERT_ICR": f"",

    # T5
    "BOSS_Toxicity_ID_T5_Insert": f"",
    "BOSS_Toxicity_ID_T5_Substitute": f"",
    "BOSS_Toxicity_ID_T5_Translate": f"",
    "BOSS_Toxicity_ID_T5_Paraphrase": f"",
    "BOSS_Toxicity_ID_T5_ICR": f"",

    # Falcon
    "BOSS_Toxicity_ID_Falcon_Insert": f"",
    "BOSS_Toxicity_ID_Falcon_Substitute": f"",
    "BOSS_Toxicity_ID_Falcon_Translate": f"",
    "BOSS_Toxicity_ID_Falcon_Paraphrase": f"",
    "BOSS_Toxicity_ID_Falcon_ICR": f"",

    # Toxigen
    # BERT
    "BOSS_Toxicity_Toxigen_BERT_Insert": f"",
    "BOSS_Toxicity_Toxigen_BERT_Substitute": f"",
    "BOSS_Toxicity_Toxigen_BERT_Translate": f"",
    "BOSS_Toxicity_Toxigen_BERT_Paraphrase": f"",
    "BOSS_Toxicity_Toxigen_BERT_ICR": f"",

    # T5
    "BOSS_Toxicity_Toxigen_T5_Insert": f"",
    "BOSS_Toxicity_Toxigen_T5_Substitute": f"",
    "BOSS_Toxicity_Toxigen_T5_Translate": f"",
    "BOSS_Toxicity_Toxigen_T5_Paraphrase": f"",
    "BOSS_Toxicity_Toxigen_T5_ICR": f"",

    # Falcon
    "BOSS_Toxicity_Toxigen_Falcon_Insert": f"",
    "BOSS_Toxicity_Toxigen_Falcon_Substitute": f"",
    "BOSS_Toxicity_Toxigen_Falcon_Translate": f"",
    "BOSS_Toxicity_Toxigen_Falcon_Paraphrase": f"",
    "BOSS_Toxicity_Toxigen_Falcon_ICR": f"",

    # ImplicitHate
    # BERT
    "BOSS_Toxicity_ImplicitHate_BERT_Insert": f"",
    "BOSS_Toxicity_ImplicitHate_BERT_Substitute": f"",
    "BOSS_Toxicity_ImplicitHate_BERT_Translate": f"",
    "BOSS_Toxicity_ImplicitHate_BERT_Paraphrase": f"",
    "BOSS_Toxicity_ImplicitHate_BERT_ICR": f"",

    # T5
    "BOSS_Toxicity_ImplicitHate_T5_Insert": f"",
    "BOSS_Toxicity_ImplicitHate_T5_Substitute": f"",
    "BOSS_Toxicity_ImplicitHate_T5_Translate": f"",
    "BOSS_Toxicity_ImplicitHate_T5_Paraphrase": f"",
    "BOSS_Toxicity_ImplicitHate_T5_ICR": f"",

    # Falcon
    "BOSS_Toxicity_ImplicitHate_Falcon_Insert": f"",
    "BOSS_Toxicity_ImplicitHate_Falcon_Substitute": f"",
    "BOSS_Toxicity_ImplicitHate_Falcon_Translate": f"",
    "BOSS_Toxicity_ImplicitHate_Falcon_Paraphrase": f"",
    "BOSS_Toxicity_ImplicitHate_Falcon_ICR": f"",

    # AdvCivil
    # BERT
    "BOSS_Toxicity_AdvCivil_BERT_Insert": f"",
    "BOSS_Toxicity_AdvCivil_BERT_Substitute": f"",
    "BOSS_Toxicity_AdvCivil_BERT_Translate": f"",
    "BOSS_Toxicity_AdvCivil_BERT_Paraphrase": f"",
    "BOSS_Toxicity_AdvCivil_BERT_ICR": f"",

    # T5
    "BOSS_Toxicity_AdvCivil_T5_Insert": f"",
    "BOSS_Toxicity_AdvCivil_T5_Substitute": f"",
    "BOSS_Toxicity_AdvCivil_T5_Translate": f"",
    "BOSS_Toxicity_AdvCivil_T5_Paraphrase": f"",
    "BOSS_Toxicity_AdvCivil_T5_ICR": f"",

    # Falcon
    "BOSS_Toxicity_AdvCivil_Falcon_Insert": f"",
    "BOSS_Toxicity_AdvCivil_Falcon_Substitute": f"",
    "BOSS_Toxicity_AdvCivil_Falcon_Translate": f"",
    "BOSS_Toxicity_AdvCivil_Falcon_Paraphrase": f"",
    "BOSS_Toxicity_AdvCivil_Falcon_ICR": f"",

    # AG News Tweets
    # ID
    # BERT
    "AgNewsTweets_ID_BERT_Insert": f"",
    "AgNewsTweets_ID_BERT_Substitute": f"",
    "AgNewsTweets_ID_BERT_Translate": f"",
    "AgNewsTweets_ID_BERT_Paraphrase": f"",
    "AgNewsTweets_ID_BERT_ICR": f"",

    # T5
    "AgNewsTweets_ID_T5_Insert": f"",
    "AgNewsTweets_ID_T5_Substitute": f"",
    "AgNewsTweets_ID_T5_Translate": f"",
    "AgNewsTweets_ID_T5_Paraphrase": f"",
    "AgNewsTweets_ID_T5_ICR": f"",

    # Falcon
    "AgNewsTweets_ID_Falcon_Insert": f"",
    "AgNewsTweets_ID_Falcon_Substitute": f"",
    "AgNewsTweets_ID_Falcon_Translate": f"",
    "AgNewsTweets_ID_Falcon_Paraphrase": f"",
    "AgNewsTweets_ID_Falcon_ICR": f"",

    # Tweets
    # BERT
    "AgNewsTweets_Tweets_BERT_Insert": f"",
    "AgNewsTweets_Tweets_BERT_Substitute": f"",
    "AgNewsTweets_Tweets_BERT_Translate": f"",
    "AgNewsTweets_Tweets_BERT_Paraphrase": f"",
    "AgNewsTweets_Tweets_BERT_ICR": f"",

    # T5
    "AgNewsTweets_Tweets_T5_Insert": f"",
    "AgNewsTweets_Tweets_T5_Substitute": f"",
    "AgNewsTweets_Tweets_T5_Translate": f"",
    "AgNewsTweets_Tweets_T5_Paraphrase": f"",
    "AgNewsTweets_Tweets_T5_ICR": f"",

    # Falcon
    "AgNewsTweets_Tweets_Falcon_Insert": f"",
    "AgNewsTweets_Tweets_Falcon_Substitute": f"",
    "AgNewsTweets_Tweets_Falcon_Translate": f"",
    "AgNewsTweets_Tweets_Falcon_Paraphrase": f"",
    "AgNewsTweets_Tweets_Falcon_ICR": f"",
}

print(json.dumps(table_paths, indent=4))

In [ ]:
run = wandb.init()

In [ ]:
def get_local_logs_path(directory):
    for contents in os.listdir(directory):
        if contents.endswith("style_logs.table.json"):
            return os.path.join(directory, contents)

        if os.path.isdir(os.path.join(directory, contents)):
            return get_local_logs_path(os.path.join(directory, contents))

print(get_local_logs_path("data/BOSS_Sentiment_Dynasent_BERT_Insert"))

In [ ]:
# for each table path, run.use_artifact and download the artifact.
local_paths = {}
missing_tables = []
for table_name, table_path in tqdm(list(table_paths.items())[:]):
    if table_path == "":
        missing_tables.append(table_name)
        continue

    download_path = f"{project_name}/{table_path}"
    print(f"Downloading {table_name} --- {download_path}")
    artifact = run.use_artifact(download_path, type="run_table")
    artifact_dir = artifact.download(root=f"data/{table_name}")
    file_name = os.listdir(artifact_dir)[0]
    local_paths[table_name] = get_local_logs_path(f"data/{table_name}")

print(f"\nMissing tables: {missing_tables}")
print(json.dumps(local_paths, indent=4))

In [ ]:
column_renaming = {
    "entropy": "tta_prediction_entropy",
    "mean probs": "tta_mean_class_probs",
    "all probs": "tta_all_class_probs",
    "all entropies": "tta_all_class_entropy",
    "latency": "tta_inference_latency",
    "input": "augmentations",
    "original_input": "original_text",
    "judgment": "tta_predicted_class",
    "original judgment": "original_predicted_class",
    "original entropy": "original_prediction_entropy",
    "entropy decrease": "prediction_entropy_decrease",
    "entropy decreased": "prediction_entropy_decreased",
}
columns_to_drop = ["style prompt", "mean exemplar distance", "prompt"]
columns_order = ["outcome", "original_text", "augmentations", "original_predicted_class", "tta_predicted_class", "label", "tta_inference_latency", "original_prediction_entropy", "tta_prediction_entropy", "prediction_entropy_decreased", "prediction_entropy_decrease", "tta_mean_class_probs", "tta_all_class_probs", "tta_all_class_entropy", ]

In [ ]:
def parse_augmentations(augmentations):
    if augmentations is None or len(augmentations) == 0:
        return []

    return [aug.replace("<aug>", "").replace("</aug>", "") for aug in augmentations[:4]]

formatted_augs = parse_augmentations([ "<aug>too not much of anything this well - well acted but dangerously slow thriller feels somehow like simply a terrible preamble to a bigger, much more too complicated horror story, one reality that never materializes.</aug>", "<aug>one too too much apprehension of this well - being acted but dangerously still slow thriller feels like only a preamble to a seemingly bigger, yet more more complicated story, and one that never materializes.</aug>", "<aug>too much of this a well - well acted but dangerously slow thriller film feels like a big preamble opening to a somehow bigger, more complicated dream story, the one concept that almost never materializes.</aug>", "<aug>too am much of this this well - acted but dangerously slow paced thriller feels somehow like a prime preamble to creating a bigger, more painfully complicated story, but one one that never ever materializes.</aug>", "<aug>Too much of this well-acted but dangerously slow thriller feels like a preamble to a bigger , more complicated story , one that never materializes .</aug>", "<aug>Too much of this well-acted but dangerously slow thriller feels like a preamble to a bigger , more complicated story , one that never materializes .</aug>" ])

print(formatted_augs)
print(len(formatted_augs))

In [ ]:
combined_datset = DatasetDict()
for split in tqdm(local_paths):
    # print(f"Processing {split}")
    path = local_paths[split]
    json_logs = json.load(open(path))
    frame = pd.DataFrame(data=json_logs["data"], columns=json_logs["columns"])
    frame = frame.rename(columns=column_renaming)
    frame = frame.drop(columns=columns_to_drop)
    frame["augmentations"] = frame["augmentations"].apply(parse_augmentations)

    if "BERT" not in split:
        for col in columns_order:
            if col not in frame.columns:
                if col == "original_prediction_entropy" or col == "tta_prediction_entropy" or col == "prediction_entropy_decrease":
                    frame[col] = float("nan")
                elif col == "prediction_entropy_decreased":
                    frame[col] = False
                elif col == "tta_all_class_probs":
                    frame[col] = frame.apply(lambda x: [[float(0.0)]], axis=1)
                elif "probs" in col:
                    frame[col] = frame.apply(lambda x: [float(0.0)], axis=1)
                elif col == "tta_all_class_entropy":
                    frame[col] = frame.apply(lambda x: [float(0.0)], axis=1)


    frame = frame[columns_order]
    combined_datset[split] = Dataset.from_pandas(frame)

combined_datset

In [ ]:
combined_datset.push_to_hub("LLM-TTA-Augmentation-Logs")